<center><h1>Thai_Khang_Final_Project</h1></center>

Name: Khang Thai
<br>
Github Username: kunfupen
<br>
USC ID: 5721113147

## 1. Transfer Learning for Image Classification

Import packages

### (a) In this problem, we are trying to build a classifier that distinguishes images of 17 types of jute pest.

### (b) Data Exploration and Pre-processing

#### (i)